### Unit 1 Homework:  Scraping the Yelp Website

Welcome!  For this homework assignment you'll be tasked with building a web scraper in a manner that builds on what was covered in our web scraping class.

The assignment will extend the lab work done during that time, where we built a dataset that listed the name, number of reviews and price range for restaurant on the following web page: https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1

**What You'll Turn In:**

A finished jupyter notebook that walks us through the steps you took in order to get your results.  Provide notes where appropriate to explain what you are doing.

The notebook should produce a finished dataset at the end.  

If for some reason you're experiencing problems with the final result, please let someone know when turning it in.
 
The homework is divided into five tiers, each of which have increasing levels of difficulty:

##### Tier 1: Five Columns From the First Page

At the most basic level for this assignment, you will need to extend what we did in class, and create a dataset that has five columns in it that are 30 rows long.  This means you will not need to go off the first page in order to complete this section.

##### Tier 2:  100 Row Dataset With At Least 3 Columns

For this portion of the assignment, take 3 of your columns from step 1, and extend them out to multiple pages on the yelp website.  You should appropriately account for the presence of missing values.

##### Tier 3:  100 Row Dataset With At Least 5 Columns

Very similar to Tier 2, but if you use this many columns you will be forced to encounter some columns that will frequently have missing values, whereas with Tier 2 you could likely skip these if you wanted to.  

##### Tier 4:  100 Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Restaurants often have different categories associated with them, so grabbing them individually as separate values is often challenging.  To complete this tier, you'll have to find a way to 'pick out' each of the individual categories as their own separate column value.  

##### Tier 5:  Unlimited Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Take what you did in Tier 4, and extend it so that the code will work with an arbitrary number of pages.  Ie, regardless of how many pages there are listing the best restaurants in London, your scraper will find them, and cleanly parse their information into clean datasets.

### Hints

Here are a few tips that will save you time when completing this assignment:

 - The name, average rating, total ratings and neighborhood of a restaurant tend to be the 'easy' ones, because they rarely have missing values, so what ever logic you use on the first page will typically apply to all pages.  They are a good place to start
 - Phone numbers, price ranges and reviews are more commonly missing, so if you are trying to get a larger number of items from them across multiple pages you should expect to do some error handling
 - You can specify any sort of selector when using the `find_all()` method, not just `class`.  For example, imagine you have the following `<div>` tag:
    `<div class='main-container red-blue-green' role='front-unit' aria-select='left-below'>Some content here</div>`
    
   This means that when you use `scraper.find_all('div')`, you can pass in arguments like `scraper.find_all('div', {'role': 'front-unit'})` or anything else that allows you to isolate that particular tag.
 - When specifying selectors like `{'class': 'dkght__384Ko'}`, sometimes less is more.  If you include multiple selectors, you are saying return a tag with **any one of these** distinctions, not all of them.  So if your results are large, try different combinations of selectors to get the smallest results possible.
 - If you begin dealing with values that are unreliably entered, you should use the 'outside in' technique where you grab a parent container that holds the element and find a way to check to see if a particular value is there by scraping it further.  The best way to do this is to try and find a unique container for every single restaurant.  This means that you will have a reliable parent element for every single restaurant, and within *each of these* you can search for `<p>`, `<a>`, `<div>`, and `<span>` tags and apply further logic.
 - When you get results from `BeautifulSoup`, you will be given data that's denoted as either `bs4.element.Tag` or `bs4.element.ResultSet`.  They are **not the same**.  Critically, you can search a `bs4.element.Tag` for further items, but you cannot do this with a `bs4.element.ResultSet`.  
 
   For example, let's say you grab all of the divs from a page with `scraper.find_all('div')` and save it as the variable `total_divs`.  This means `total_divs` will look somethig like this:  
   
   `[<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>]`
      
   In this case the variable `total_divs` is a result set and there's nothing else you can do to it directly.  However, every item within `total_divs` is a tag, which means you can scrape it further.  
   
   So if you wanted you could write a line like:  `total_paragraphs = [div.find_all('p') for div in total_divs]`, and get the collection of paragraphs within each div.  
   
   If you confuse the two you'll get the following error message:  
   
   `AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?`
 - The values of the different selectors change periodically on yelp, so if your scraper all of a sudden stops working that's probably why.  Ie, if you have a command like `scraper.find_all('div', {'class': '485dk0W__container09'}` that no longer returns results, the class `485dk0W__container09` may now be `r56kW__container14` or something similar.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#scrape multiple pages
page_ranges = [0,10,20]

#create new lists for scraped data
all_names = []
num_reviews  = []
price_ranges = []
all_paragraphs = []
num_stars = []

#allow scraping to search multiple pages
for i in page_ranges:
    url = (f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1&start={i}')
    yelp_req= requests.get(url).text
    scraper= BeautifulSoup(yelp_req)
    #scrape names of each restaurant
    names=scraper.find_all('a',{'class': 'css-166la90'})
    names = [name.text for name in names]
    names=[name for name in names if len(name)>1]
    all_names += names
    #scrape number of reviews for each restaurant
    reviews=scraper.find_all('span', {'class':'reviewCount__09f24__EUXPN'})
    reviews = [int(review.text) for review in reviews]
    num_reviews += reviews
    #scrape price range for each restaurant
    prices=scraper.find_all('span', {'class':'priceRange__09f24__2O6le css-xtpg8e'})
    prices = [str(price.text) for price in prices]
    price_ranges += prices
    #scrape the first review for each restaurant
    paragraphs=scraper.find_all('p',{'class': 'css-e81eai'})
    paragraphs = [str(paragraph.text) for paragraph in paragraphs]
    paragraphs=[paragraph for paragraph in paragraphs if len(paragraph)>1]
    paragraphs = [text.replace('\xa0more','') for text in paragraphs[:10]]
    all_paragraphs += paragraphs
    #scrape the star rating for each restaurant
    ratings=scraper.find_all('div',{'class': 'i-stars__09f24__1T6rz'})
    ratings = [float(item['aria-label'].split()[0]) for item in ratings]
    num_stars += ratings

In [169]:
#create new dictionary for each item
df_dict = {
    'Title': all_names,
    'Number of Ratings': num_reviews,
    'Price Range': price_ranges,
    'Number of Stars':num_stars,
    'First Review':all_paragraphs
}
#create table format
pd.DataFrame(df_dict)

,Title,Number of Ratings,Price Range,Number of Stars,First Review
0,The Mayfair Chippy,282,££,4.5,“One of the best fish ever with the most tasty...
1,Dishoom,1841,££,4.5,“Hard to find a way to add any higher praise t...
2,The Breakfast Club,494,££,4.0,“By far one of my most favorite breakfast plac...
3,Flat Iron,380,££,4.5,“Went to London for vacation and stopped by th...
4,Ffiona’s Restaurant,267,££,4.5,“Ffiona's is easily my favorite restaurant in ...
5,Dishoom,547,££,4.5,“I visited Dishoom during my recent London tri...
6,The Fat Bear,122,££,4.5,"“WOW, this place is delicious!\n\nOur family s..."
7,Restaurant Gordon Ramsay,204,££££,4.5,“Compared to Michelin 3-star restaurants in Ca...
8,Mother Mash,470,££,4.0,“Soho is full of culture and amazing places to...
9,NOPI,272,£££,4.5,“10/10 recommend!\nGood cocktails to have as a...
